In [1]:
pip install vaderSentiment


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
print("VADER Sentiment installed successfully!")


VADER Sentiment installed successfully!


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

# Load the scraped data
data = pd.read_csv('reddit_scraped_data.csv')  # Replace with your file name

# Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to compute sentiment scores
def analyze_sentiment(text):
    if pd.isna(text):  # Handle missing values
        return {'positive': 0, 'neutral': 0, 'negative': 0, 'compound': 0}
    scores = analyzer.polarity_scores(text)
    return scores

# Apply sentiment analysis to the text data
data['sentiment_scores'] = data['cleaned_text'].apply(analyze_sentiment)

# Extract sentiment components into separate columns
data['positive'] = data['sentiment_scores'].apply(lambda x: x['pos'])
data['neutral'] = data['sentiment_scores'].apply(lambda x: x['neu'])
data['negative'] = data['sentiment_scores'].apply(lambda x: x['neg'])
data['compound'] = data['sentiment_scores'].apply(lambda x: x['compound'])

# Categorize sentiment based on compound score
def categorize_sentiment(compound_score):
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

data['sentiment_category'] = data['compound'].apply(categorize_sentiment)

# Save the results
data.to_csv('reddit_data_with_sentiment.csv', index=False)
print("Sentiment analysis complete! Results saved to 'reddit_data_with_sentiment.csv'")


In [ ]:
#Prepare Data for the Model:
from sklearn.preprocessing import LabelEncoder

# Convert sentiment category into numerical labels
label_encoder = LabelEncoder()
data['sentiment_category_encoded'] = label_encoder.fit_transform(data['sentiment_category'])

# Combine numerical sentiment features with other features
X_sentiment = data[['positive', 'neutral', 'negative', 'compound', 'sentiment_category_encoded']]


In [ ]:
#Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Plot sentiment categories
sns.countplot(x='sentiment_category', data=data, order=['positive', 'neutral', 'negative'])
plt.title('Sentiment Category Distribution')
plt.show()
